In this journal, we aim to train a functioning audio sentiment model. This model would be utilised in an analyser, which can be represented as:

Audio Object -------> Sentiment Analyser ------> Sentiment Object

The file names for each audio file describes various details about the short audio clip that comes with it, including the emotion. We will use emotion attached to train the model.

In [4]:
import os
from fetch_emotion import get_emotion
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Activation
from keras.utils import to_categorical, np_utils
from keras.models import Sequential, Model

In [5]:
# Set number of threads
tf.config.threading.set_inter_op_parallelism_threads(8)
tf.config.threading.set_intra_op_parallelism_threads(8)

In [27]:
n_labels = 7
path = '../audiofiles'

Here, we want to have all the files available for training. We will skip non-wav files.

In [7]:
file_names = []
for path, _, files in os.walk(path):
    for name in files:
        if name[-3:] == 'wav':
            file_names.append(os.path.join(path, name))

Audio features have to be extracted from each audio sample into a data frame.

In [20]:
# Initialise dataframe with 2 columns for features of audio and emotion
df = pd.DataFrame(columns=['feature', 'emotion'])

In [21]:
for i, name in enumerate(file_names):
    # Extract data from audio file     
    X, sample_rate = librosa.load(name, res_type='kaiser_fast', duration=3, sr=22050*2, offset=0.5)
    sample_rate = np.array(sample_rate)
    # Get features of audio data and attach label
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    df.loc[i] = [mfccs, get_emotion(name)]

Preprocess into suitable dimensions.

In [46]:
feature_df = pd.DataFrame(df['feature'].values.tolist())
labelled_df = pd.concat([feature_df, df['emotion']], axis=1)
labelled_df = labelled_df.dropna()
shuffled_df = labelled_df.sample(frac=1).reset_index(drop=True)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(shuffled_df.drop('emotion', axis=1), shuffled_df['emotion'], test_size=0.3, random_state=17, shuffle = True)

Now, we want to encode the labels for the training and test data locally.

In [60]:
encoder = LabelEncoder()
y_train_cat = np_utils.to_categorical(encoder.fit_transform(y_train))
y_test_cat = np_utils.to_categorical(encoder.fit_transform(y_test))
np.save('label_classes.npy', encoder.classes_)

In [61]:
x_train_exp = np.expand_dims(X_train, axis=2)
x_test_exp = np.expand_dims(X_test, axis=2)

Training the model.

In [62]:
model = Sequential()

# Layer 1
model.add(Conv1D(256, 5, padding='same', input_shape=(x_train_exp.shape[1], 1)))
model.add(Activation('relu'))

# layer 2
model.add(Conv1D(128, 5, padding='same'))
model.add(Activation('relu'))

# Regularisation and Pooling
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=(8)))

# Layer 3
model.add(Conv1D(128, 5, padding='same'))
model.add(Activation('relu'))

# Converge Nodes and conversion to probability distribution
model.add(Flatten())
model.add(Dense(n_labels)) 
model.add(Activation('softmax')) 

opt = keras.optimizers.RMSprop(learning_rate=0.0001, weight_decay=1e-6)

In [63]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [64]:
history = model.fit(x_train_exp, y_train_cat, batch_size=128, epochs=600, validation_data = [x_test_exp, y_test_cat])

Epoch 1/600
14/14 [==============================] - 3s 163ms/step - loss: 3.9716 - accuracy: 0.1865 - val_loss: 1.9339 - val_accuracy: 0.1916
Epoch 2/600
14/14 [==============================] - 2s 153ms/step - loss: 2.8289 - accuracy: 0.2139 - val_loss: 1.8959 - val_accuracy: 0.2486
Epoch 3/600
14/14 [==============================] - 2s 165ms/step - loss: 2.5146 - accuracy: 0.2197 - val_loss: 2.3716 - val_accuracy: 0.2337
Epoch 4/600
14/14 [==============================] - 2s 152ms/step - loss: 2.3328 - accuracy: 0.2383 - val_loss: 1.7845 - val_accuracy: 0.2717
Epoch 5/600
14/14 [==============================] - 2s 152ms/step - loss: 2.1810 - accuracy: 0.2383 - val_loss: 1.9692 - val_accuracy: 0.3152
Epoch 6/600
14/14 [==============================] - 2s 154ms/step - loss: 2.0574 - accuracy: 0.2541 - val_loss: 1.6783 - val_accuracy: 0.3288
Epoch 7/600
14/14 [==============================] - 2s 156ms/step - loss: 1.9805 - accuracy: 0.2867 - val_loss: 2.0291 - val_accuracy: 0.2908

Epoch 58/600
14/14 [==============================] - 2s 155ms/step - loss: 1.2143 - accuracy: 0.5484 - val_loss: 1.2722 - val_accuracy: 0.5367
Epoch 59/600
14/14 [==============================] - 2s 154ms/step - loss: 1.1709 - accuracy: 0.5688 - val_loss: 1.3914 - val_accuracy: 0.4402
Epoch 60/600
14/14 [==============================] - 2s 152ms/step - loss: 1.1987 - accuracy: 0.5629 - val_loss: 1.2447 - val_accuracy: 0.5693
Epoch 61/600
14/14 [==============================] - 2s 145ms/step - loss: 1.1946 - accuracy: 0.5635 - val_loss: 1.3718 - val_accuracy: 0.5122
Epoch 62/600
14/14 [==============================] - 2s 163ms/step - loss: 1.1756 - accuracy: 0.5682 - val_loss: 1.2980 - val_accuracy: 0.5448
Epoch 63/600
14/14 [==============================] - 2s 139ms/step - loss: 1.1699 - accuracy: 0.5612 - val_loss: 1.3616 - val_accuracy: 0.4932
Epoch 64/600
14/14 [==============================] - 2s 152ms/step - loss: 1.1681 - accuracy: 0.5670 - val_loss: 1.3048 - val_accuracy:

Epoch 115/600
14/14 [==============================] - 2s 154ms/step - loss: 0.8885 - accuracy: 0.6841 - val_loss: 1.0819 - val_accuracy: 0.5951
Epoch 116/600
14/14 [==============================] - 2s 155ms/step - loss: 0.8676 - accuracy: 0.6906 - val_loss: 1.1039 - val_accuracy: 0.5951
Epoch 117/600
14/14 [==============================] - 2s 160ms/step - loss: 0.8781 - accuracy: 0.6900 - val_loss: 1.0658 - val_accuracy: 0.6182
Epoch 118/600
14/14 [==============================] - 2s 159ms/step - loss: 0.8674 - accuracy: 0.6941 - val_loss: 1.0801 - val_accuracy: 0.6196
Epoch 119/600
14/14 [==============================] - 2s 138ms/step - loss: 0.8627 - accuracy: 0.6935 - val_loss: 1.0635 - val_accuracy: 0.6209
Epoch 120/600
14/14 [==============================] - 2s 153ms/step - loss: 0.8699 - accuracy: 0.6812 - val_loss: 1.0244 - val_accuracy: 0.6562
Epoch 121/600
14/14 [==============================] - 2s 147ms/step - loss: 0.8375 - accuracy: 0.7075 - val_loss: 1.0843 - val_ac

14/14 [==============================] - 2s 161ms/step - loss: 0.6705 - accuracy: 0.7716 - val_loss: 1.0177 - val_accuracy: 0.6386
Epoch 172/600
14/14 [==============================] - 2s 151ms/step - loss: 0.6719 - accuracy: 0.7675 - val_loss: 1.0027 - val_accuracy: 0.6508
Epoch 173/600
14/14 [==============================] - 2s 158ms/step - loss: 0.6937 - accuracy: 0.7582 - val_loss: 1.0799 - val_accuracy: 0.6033
Epoch 174/600
14/14 [==============================] - 2s 158ms/step - loss: 0.6599 - accuracy: 0.7628 - val_loss: 0.9922 - val_accuracy: 0.6454
Epoch 175/600
14/14 [==============================] - 2s 158ms/step - loss: 0.6648 - accuracy: 0.7605 - val_loss: 0.9439 - val_accuracy: 0.6522
Epoch 176/600
14/14 [==============================] - 2s 160ms/step - loss: 0.6554 - accuracy: 0.7646 - val_loss: 0.9581 - val_accuracy: 0.6467
Epoch 177/600
14/14 [==============================] - 2s 154ms/step - loss: 0.6366 - accuracy: 0.7745 - val_loss: 1.0228 - val_accuracy: 0.6291

14/14 [==============================] - 2s 154ms/step - loss: 0.5156 - accuracy: 0.8223 - val_loss: 0.9461 - val_accuracy: 0.6576
Epoch 228/600
14/14 [==============================] - 2s 149ms/step - loss: 0.5181 - accuracy: 0.8193 - val_loss: 0.8468 - val_accuracy: 0.7065
Epoch 229/600
14/14 [==============================] - 2s 157ms/step - loss: 0.5161 - accuracy: 0.8228 - val_loss: 0.8368 - val_accuracy: 0.7106
Epoch 230/600
14/14 [==============================] - 2s 164ms/step - loss: 0.4988 - accuracy: 0.8205 - val_loss: 0.9075 - val_accuracy: 0.6753
Epoch 231/600
14/14 [==============================] - 2s 163ms/step - loss: 0.5062 - accuracy: 0.8281 - val_loss: 0.9819 - val_accuracy: 0.6630
Epoch 232/600
14/14 [==============================] - 2s 166ms/step - loss: 0.5089 - accuracy: 0.8159 - val_loss: 0.8519 - val_accuracy: 0.6848
Epoch 233/600
14/14 [==============================] - 2s 157ms/step - loss: 0.5013 - accuracy: 0.8211 - val_loss: 0.9081 - val_accuracy: 0.6726

...

14/14 [==============================] - 2s 157ms/step - loss: 0.3058 - accuracy: 0.9015 - val_loss: 0.7695 - val_accuracy: 0.7201
Epoch 340/600
14/14 [==============================] - 2s 159ms/step - loss: 0.3040 - accuracy: 0.9009 - val_loss: 0.7378 - val_accuracy: 0.7405
Epoch 341/600
14/14 [==============================] - 2s 146ms/step - loss: 0.3112 - accuracy: 0.8928 - val_loss: 0.8227 - val_accuracy: 0.7188
Epoch 342/600
14/14 [==============================] - 2s 152ms/step - loss: 0.3024 - accuracy: 0.9068 - val_loss: 0.9096 - val_accuracy: 0.6821
Epoch 343/600
14/14 [==============================] - 2s 167ms/step - loss: 0.3200 - accuracy: 0.8928 - val_loss: 0.7954 - val_accuracy: 0.7215
Epoch 344/600
14/14 [==============================] - 2s 169ms/step - loss: 0.2942 - accuracy: 0.8974 - val_loss: 0.7410 - val_accuracy: 0.7405
Epoch 345/600
14/14 [==============================] - 2s 146ms/step - loss: 0.2933 - accuracy: 0.8969 - val_loss: 0.7732 - val_accuracy: 0.7160

14/14 [==============================] - 2s 156ms/step - loss: 0.2294 - accuracy: 0.9178 - val_loss: 0.7699 - val_accuracy: 0.7242
Epoch 396/600
14/14 [==============================] - 2s 156ms/step - loss: 0.2413 - accuracy: 0.9254 - val_loss: 0.7456 - val_accuracy: 0.7323
Epoch 397/600
14/14 [==============================] - 2s 158ms/step - loss: 0.2352 - accuracy: 0.9231 - val_loss: 0.7382 - val_accuracy: 0.7351
Epoch 398/600
14/14 [==============================] - 2s 156ms/step - loss: 0.2316 - accuracy: 0.9219 - val_loss: 0.7268 - val_accuracy: 0.7527
Epoch 399/600
14/14 [==============================] - 2s 166ms/step - loss: 0.2140 - accuracy: 0.9318 - val_loss: 0.7567 - val_accuracy: 0.7378
Epoch 400/600
14/14 [==============================] - 2s 150ms/step - loss: 0.2295 - accuracy: 0.9283 - val_loss: 0.7519 - val_accuracy: 0.7527
Epoch 401/600
14/14 [==============================] - 2s 158ms/step - loss: 0.2234 - accuracy: 0.9307 - val_loss: 0.8215 - val_accuracy: 0.7351

14/14 [==============================] - 2s 155ms/step - loss: 0.1821 - accuracy: 0.9388 - val_loss: 0.7607 - val_accuracy: 0.7405
Epoch 452/600
14/14 [==============================] - 2s 161ms/step - loss: 0.1746 - accuracy: 0.9435 - val_loss: 0.9169 - val_accuracy: 0.7092
Epoch 453/600
14/14 [==============================] - 2s 151ms/step - loss: 0.1834 - accuracy: 0.9411 - val_loss: 0.7152 - val_accuracy: 0.7418
Epoch 454/600
14/14 [==============================] - 2s 158ms/step - loss: 0.1828 - accuracy: 0.9458 - val_loss: 0.7522 - val_accuracy: 0.7418
Epoch 455/600
14/14 [==============================] - 2s 167ms/step - loss: 0.1809 - accuracy: 0.9481 - val_loss: 0.8094 - val_accuracy: 0.7310
Epoch 456/600
14/14 [==============================] - 2s 154ms/step - loss: 0.1659 - accuracy: 0.9516 - val_loss: 0.7109 - val_accuracy: 0.7527
Epoch 457/600
14/14 [==============================] - 2s 151ms/step - loss: 0.1738 - accuracy: 0.9446 - val_loss: 0.7205 - val_accuracy: 0.7473

14/14 [==============================] - 2s 152ms/step - loss: 0.1312 - accuracy: 0.9604 - val_loss: 0.8008 - val_accuracy: 0.7432
Epoch 508/600
14/14 [==============================] - 2s 158ms/step - loss: 0.1466 - accuracy: 0.9586 - val_loss: 0.7536 - val_accuracy: 0.7568
Epoch 509/600
14/14 [==============================] - 2s 155ms/step - loss: 0.1438 - accuracy: 0.9534 - val_loss: 0.7595 - val_accuracy: 0.7514
Epoch 510/600
14/14 [==============================] - 2s 169ms/step - loss: 0.1311 - accuracy: 0.9575 - val_loss: 0.7786 - val_accuracy: 0.7323
Epoch 511/600
14/14 [==============================] - 2s 161ms/step - loss: 0.1426 - accuracy: 0.9528 - val_loss: 0.8386 - val_accuracy: 0.7446
Epoch 512/600
14/14 [==============================] - 2s 149ms/step - loss: 0.1318 - accuracy: 0.9639 - val_loss: 0.7516 - val_accuracy: 0.7568
Epoch 513/600
14/14 [==============================] - 2s 149ms/step - loss: 0.1381 - accuracy: 0.9586 - val_loss: 0.7352 - val_accuracy: 0.7486

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Trainig Completed. Display results.

In [65]:
train_result = model.evaluate(x_train_exp, y_train_cat, verbose=0)
test_result = model.evaluate(x_test_exp, y_test_cat, verbose=0)

In [66]:
print("Train", "%s: %.2f%%" % (model.metrics_names[1], train_result[1] * 100))
print("Test", "%s: %.2f%%" % (model.metrics_names[1], test_result[1] * 100))

Train accuracy: 99.18%
Test accuracy: 74.59%


We have now finished training the model and so we can save and start using it.

In [14]:
model_name = 'Audio_Sentiment_Model.h5'
save_dir = os.getcwd()
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

Having completed training, we can have a look at how the model performs and find any information that is useful to note. 

In [44]:
# Get Predictions
preds = model.predict(x_test_exp, batch_size=512, verbose=1)
preds_mod = preds.argmax(axis=1)
preds_flat = preds_mod.astype(int).flatten()
preds_transformed = (encoder.inverse_transform((preds_flat)))
pred_df = pd.DataFrame({'predicted': preds_transformed})

# Get Actual Values
actual_values = y_test_cat.argmax(axis=1)
actual_values_mod = actual_values.astype(int).flatten()
actual_values_mod_transformed = (encoder.inverse_transform((actual_values_mod)))
actual_df = pd.DataFrame({'actual': actual_values_mod_transformed})
final_df = actual_df.join(pred_df)

2/2 [==============================] - 0s 21ms/step


In [ ]:
# TODO